## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-26-18-50-02 +0000,startribune,Taylor Swift and Travis Kelce are engaged,https://www.startribune.com/taylor-swift-and-t...
1,2025-08-26-18-49-56 +0000,bbc,Taylor Swift and Travis Kelce announce engagement,https://www.bbc.com/news/articles/c707grqqdnwo...
2,2025-08-26-18-49-53 +0000,startribune,"Live: Who’s in, who’s out for the Vikings on r...",https://www.startribune.com/minnesota-vikings-...
3,2025-08-26-18-48-15 +0000,48hills,Screen Grabs: Red Sonja rides again,https://48hills.org/2025/08/screen-grabs-red-s...
4,2025-08-26-18-46-00 +0000,wsj,Trump’s Media Company Launches New Business to...,https://www.wsj.com/finance/currencies/trumps-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2334/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
18,trump,71
33,cook,18
22,new,18
86,gaza,16
32,lisa,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
194,2025-08-26-03-44-00 +0000,wsj,"President Trump said he is removing Lisa Cook,...",https://www.wsj.com/politics/policy/trump-says...,179
161,2025-08-26-09-30-00 +0000,wsj,President Trump’s attempt to fire Federal Rese...,https://www.wsj.com/economy/central-banking/un...,157
69,2025-08-26-16-22-37 +0000,nyt,Can Trump Fire a Fed Governor? What to Know as...,https://www.nytimes.com/2025/08/22/us/politics...,154
75,2025-08-26-16-12-46 +0000,wapo,Federal Reserve governor Lisa Cook says she’ll...,https://www.washingtonpost.com/business/2025/0...,149
68,2025-08-26-16-22-48 +0000,nyt,Lisa Cook Defies Trump Bid to Fire Her as Fed ...,https://www.nytimes.com/2025/08/26/us/politics...,147


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
194,179,2025-08-26-03-44-00 +0000,wsj,"President Trump said he is removing Lisa Cook,...",https://www.wsj.com/politics/policy/trump-says...
276,60,2025-08-25-20-38-55 +0000,nypost,Chicago’s far-left mayor calls locking up crim...,https://nypost.com/2025/08/25/us-news/illinois...
17,53,2025-08-26-18-22-31 +0000,nyt,Israel Links Deadly Hospital Attack in Gaza to...,https://www.nytimes.com/2025/08/26/world/middl...
140,50,2025-08-26-10-39-53 +0000,cbc,Australia accuses Iran of being behind two ant...,https://www.cbc.ca/news/world/australia-iran-a...
297,49,2025-08-25-19-40-01 +0000,nypost,Trump claims ‘good conversation’ with Putin si...,https://nypost.com/2025/08/25/us-news/trump-cl...
98,38,2025-08-26-15-07-12 +0000,nypost,Killer dad Chris Watts’ twisted prison letters...,https://nypost.com/2025/08/26/us-news/killer-d...
59,35,2025-08-26-16-37-38 +0000,nypost,Trump threatens more tariffs on nations that t...,https://nypost.com/2025/08/26/business/trump-t...
197,35,2025-08-26-03-05-59 +0000,nypost,American tourist stabbed while defending women...,https://nypost.com/2025/08/25/world-news/ameri...
15,34,2025-08-26-18-25-00 +0000,nypost,Hakeem Jeffries ripped for noting Lisa Cook’s ...,https://nypost.com/2025/08/26/us-news/hakeem-j...
293,33,2025-08-25-19-53-56 +0000,nypost,Kilmar Abrego Garcia cannot be deported — for ...,https://nypost.com/2025/08/25/us-news/kilmar-a...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
